In [ ]:
import astropy.coordinates as coord
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import matplotlib.pyplot as plt

In [ ]:
a = Angle(149.7943, u.degree)
b = Angle(-4.8083, u.degree)
d = 54
lon = a.radian
lat = b.radian
coord.spherical_to_cartesian(d,lat,lon)

gc = SkyCoord(l=74.305*u.degree, b = 79.822*u.degree,frame='galactic')
gc.transform_to('supergalactic')
a = gc


In [ ]:
coord_alpha = [83.61, 272.46, 276.40, 279.77, 280.72, 282.59, 286.91, 304.95, 307.74]
coord_dec = [22.00, -19.34, -13.37, -5.71, -3.51, 0.14, 6.32, 36.78, 41.23]

In [ ]:
galLon = []
galLat = []
for i,j in zip(coord_alpha, coord_dec):
    galLon.append(SkyCoord(ra=i, dec=j,
                     unit=(u.degree, u.degree)).galactic.l.value)
    galLat.append(SkyCoord(ra=i, dec=j,
                     unit=(u.degree, u.degree)).galactic.b.value)

 

In [ ]:
print(galLon)
print(galLat)

In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from math import hypot


def read_events(filename):
    # Функція для читання з файлу нових положень КП та розмазування у випадковому маг. полі
    file = open(filename, 'r')
    l = []
    b = []
    theta = []
    for line in file.readlines():
        l.append(float(line.split()[1]))
        b.append(float(line.split()[2]))
        theta.append(float(line.split()[3]))
    return np.array([l, b, theta])


init_evs_l = np.array([ta_pao_new[i][1] for i in range(len(ta_pao_new))])  # Початкові полження КП: l
init_evs_b = np.array([ta_pao_new[i][2] for i in range(len(ta_pao_new))])  # Початкові полження КП: b



evs6 = read_events('z26_new_events_paoCalib_ta_fe.txt')  # Події і кути розмазування для z=26
evsP = read_events('z1_new_events_paoCalib_ta_fe.txt')


#_____________________________________
def circle(alpha, l0, b0):
    alpha = np.radians(alpha)
    l0 = np.radians(l0)
    b0 = np.radians(-(90 - b0))
    t = np.linspace(0, 2 * np.pi, 400)
    x = np.sin(alpha) * np.cos(b0) * np.cos(l0) * np.cos(t) + np.sin(alpha) * np.sin(l0) * np.sin(t) - np.cos(alpha) * np.sin(b0) * np.cos(l0)
    y = -np.sin(alpha) * np.cos(b0) * np.sin(l0) * np.cos(t) + np.sin(alpha) * np.cos(l0) * np.sin(t) + np.cos(alpha) * np.sin(b0) * np.sin(l0)
    z = np.sin(alpha) * np.sin(b0) * np.cos(t) + np.cos(alpha) * np.cos(b0)
    l = -np.arctan2(y, x)
    b = np.arcsin(z)
    return [l * 180 / np.pi, b * 180 / np.pi]


m = Basemap(projection='moll', lon_0=0, celestial=True, resolution='c')
# draw parallels and meridians.
m.drawparallels(np.arange(-90., 90., 15.), labels=[1, 0, 0, 0], color='k')
m.drawmeridians(np.arange(0., 360., 30.))
meridian = np.arange(0., 360., 30.)
for i in np.arange(len(meridian)):
    plt.annotate(np.str(meridian[i]) + '$^\\circ$', xy=m(meridian[i], 0), xycoords='data')


l0, b0 = m(init_evs_l, init_evs_b)
#m.scatter(l0, b0, s=20, c='k', marker='x', label='Registered cosmic ray events')


evs6_l, evs6_b = m(evs6[0], evs6[1])
#m.scatter(evs6_l[:], evs6_b[:], s=20, c='b', marker='o', label='Deflected cosmic rays')

#для протонів коли разом з гелієм
evsP_l, evsP_b = m(evsP[0], evsP[1])
#m.scatter(evsP_l[:], evsP_b[:], s=20, c='b', marker='o', label='Deflected cosmic rays')


for i in range(len(evs1[0])):
    # __________Кола розмазування у випадковому гал полі для z=26__________
    r = evs6[2][i]
    res = circle(np.sqrt(r ** 2 + 30 ** 2), evs6[0][i], evs6[1][i]) # зсув 90 градусів вже враховано всередині функціїї
                                                                    # circ(). підставляємо просто b а не 90-b
    # Розбиваємо кожне коло на дві частини: 0 < l < 180 і l > 180
    # для того, що не малювалися лінії, які з'єднують точки
    # через всю карту
    res_left = [[], []]
    res_right = [[], []]
    for i in range(len(res[0])):
        if 0 < res[0][i] < 180:
            res_left[0].append(res[0][i])
            res_left[1].append(res[1][i])
        else:
            res_right[0].append(res[0][i])
            res_right[1].append(res[1][i])
    # Кожну з двох частин кола будемо малювати відрізками між двома сусідніми точками,
    # за умови, що відстань між ними менше певної величини. Ця величина залежить від загальної кількості
    # точок кола, яка задається всередині функції circle(): t = np.linspace(0, 2 * np.pi, 700) (коло має 700 точок)
    for i in range(len(res_left[0]) - 1):
        if hypot(res_left[0][i + 1] - res_left[0][i], res_left[1][i + 1] - res_left[1][i]) < 10:
            x_l, y_l = m(res_left[0][i], res_left[1][i])
            x_l2, y_l2 = m(res_left[0][i + 1], res_left[1][i + 1])
            m.plot([x_l, x_l2], [y_l, y_l2], color='b', ls='-', lw=1, label='z=6 + extragal')
    for i in range(len(res_right[0]) - 1):
        if hypot(res_right[0][i + 1] - res_right[0][i], res_right[1][i + 1] - res_right[1][i]) < 10:
            x_r, y_r = m(res_right[0][i], res_right[1][i])
            x_r2, y_r2 = m(res_right[0][i + 1], res_right[1][i + 1])
            m.plot([x_r, x_r2], [y_r, y_r2], color='b', ls='-', lw=1)


# _________Додавання легенди до малюнка (з видалленям дублікатів однакових ліній)_________
handles, labels = plt.gca().get_legend_handles_labels()
newLabels, newHandles = [], []
for handle, label in zip(handles, labels):
    if label not in newLabels:
        newLabels.append(label)
        newHandles.append(handle)
plt.legend(newHandles, newLabels, loc='upper right', prop={'size': 6})

plt.tight_layout()

plt.show()



In [ ]:
def ang_len(x1,y1,x2,y2):
    return ((x2-x1)**2 + (y2-y1)**2)**0.5

In [ ]:
ang_len(3,3,4,4)

In [ ]:
mag_lon = [9.99592, 10.72684, 15.35266, 23.34019, 23.24974, 27.38685, 33.57855, 43.02105, 57.24674, 6.91224, 9.7364, 15.03721, 29.56445, 72.32032, 29.71195]
mag_lat = [-0.24194, -0.15839, -1.02095, 0.02042, -0.341, -0.00655, -0.04507, 0.76561, 0.81898, 0.06853, -0.26041, -0.25134, 0.10634, -0.10102, -0.24011]
HAWC_lon = [184.55825082833528, 11.06789335428678, 18.115919005783056, 26.44354119763707, 28.83295780584482, 32.93364674684601, 40.40116997827228, 75.01677094008521, 79.90121001508535]
HAWC_lat = [-5.81019557983303, 0.033042191208168724, -0.46012377048554015, 0.1352701905901796, 0.3005953936642766, 0.30366960293231876, -0.7040644056486725, 0.28326481716604796, 1.084356906171349]

In [ ]:
len(mag_lon) == len(mag_lat)

In [ ]:
table = [[0 for x in range(len(HAWC_lon))] for y in range(len(mag_lon))]
x,y = 0,0
for i,j in zip(HAWC_lon, HAWC_lat):
    #print("i = ", i, "j = ", j)
    for l,b in zip(mag_lon, mag_lat):
        #print(ang_len(i,j,l,b))

In [ ]:
def extract_TA_events(ev_TA): # зчитування подій з файла, запис їх в 3 списки - енергія, довгота і широта, і формування результуючого списку списків
    TA_events = []
    file_w_events = open(ev_TA, 'r')  # open file with events
    events_list = file_w_events.readlines()  # read all lines into list of strings
    file_w_events.close()  # close events file
    E = []
    lon = []
    lat = []
    for i in events_list:
        sep_list = i.split()
        E.append(float(sep_list[0]))
        lon.append(float(sep_list[1]))
        lat.append(float(sep_list[2]))
    TA_events.append(E)
    TA_events.append(lon)
    TA_events.append(lat)
    return TA_events


def extract_PAO_events(ev_PAO): # зчитування подій з файла, запис їх в 3 списки - енергія, довгота і широта, і формування результуючого списку списків
    PAO_events = []
    file_w_events = open(ev_PAO, 'r')  # open file with events
    events_list = file_w_events.readlines()  # read all lines into list of strings
    file_w_events.close()  # close events file
    E = []
    lon = []
    lat = []
    for i in events_list:
        sep_list = i.split()
        E.append(float(sep_list[3]))
        lon.append(float(sep_list[6]))
        lat.append(float(sep_list[7]))
    PAO_events.append(E)
    PAO_events.append(lon)
    PAO_events.append(lat)
    return PAO_events


In [ ]:
ev_TA_file = 'TA_gal_events.txt' # файл з ТА подіями
ev_PAO_file = 'PAO_events.txt'  # файл з Оже подіями
PAO = extract_PAO_events(ev_PAO_file)
print(PAO[0])

In [ ]:
from Integral import test_data

In [ ]:
neutrinos_file = open('Neutrino_data.txt')
neutrinos_list = neutrinos_file.readlines()
neutrinos_file.close()

lon = []
lat = []
RA = []
dec = []
for i in neutrinos_list:
        sep_list = i.split()
        #E.append(float(sep_list[3]))
        dec.append(float(sep_list[6]))
        RA.append(float(sep_list[7]))
        
#for i in range(len(neutrinos_list)):
#    print('_____________',i)
#    print(RA[i])
#    print(dec[i])
        
#print('RA = ', min(RA))
#print('dec = ', min(dec))
    
#print(dec[2])
#neutrinos_eq = [RA, dec]

for i in range(len(dec)):
   # print(i)
    #print(dec[i])
    #print(RA[i])
    temp_coord = SkyCoord(RA[i], dec[i], frame='icrs', unit='deg') 
    lon.append(temp_coord.galactic.l.deg)
    lat.append(temp_coord.galactic.b.deg)



#print(neutrinos_eq[1][1])
#print(neutrinos_eq[0][1])
test_coord = SkyCoord(RA[0]*u.deg, dec[0]*u.deg, frame = 'icrs')

print(test_coord.galactic.l.deg)
print(test_coord.galactic.b.deg)
#print(lon)
#print(lat)

neutrinos_gal = [lon, lat]
print(neutrinos_gal[0][0],neutrinos_gal[1][0])


In [ ]:
gal_cluster_data = open('Abels.txt')
gal_cluster_list = gal_cluster_data.readlines()
gal_cluster_data.close()

lon = []
lat = []
for i in gal_cluster_list:
        sep_list = i.split()
        lon.append(float(sep_list[1]))
        lat.append(float(sep_list[2]))

gal_clusters = [lon, lat]

In [ ]:
shapley_gal = SkyCoord(311.9*u.deg, 30.73*u.deg, frame='galactic')
shapley_gal.transform_to('icrs')
SLOAN_coord = SkyCoord(172.5*u.deg,25.5*u.deg, frame='icrs')
BOSS_coord = SkyCoord(162*u.deg,51*u.deg, frame='icrs')
BOSS_coord.transform_to('galactic')


In [ ]:
# 135.5
test_coord = SkyCoord(288.3*u.deg,0.34*u.deg, frame='icrs')
test_coord.transform_to('galactic')

In [ ]:
#101.4
test_coord = SkyCoord(285.74*u.deg,-1.69*u.deg, frame='icrs')
test_coord.transform_to('galactic')